In [1]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import random
from tqdm.notebook import tqdm
plt.rcParams['figure.figsize'] = [10, 10]

np.set_printoptions(suppress=True)

In [2]:
def read_image(path, size = None):
    img = cv2.imread(path)
    if size:
        img_size = img.shape
        img = cv2.resize(img, dsize = None, fx = size[0] / img_size[0], fy = size[0] / img_size[0])
    img = cv2.flip(img, 1)
    img_gray= cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
    return img_gray, img, img_rgb

In [3]:
def recover_color(pic):
    pic = pic * 255
    b, g, r = cv2.split(pic)
    pic = cv2.merge([r, g, b])
    return pic

def hex_color(a):
    return a[0] * 256 * 256 + a[1] * 256 + a[2]

In [4]:
def SIFT(img):
    siftDetector= cv2.SIFT_create()
    kp, des = siftDetector.detectAndCompute(img, None)
    return kp, des

def plot_sift(gray, rgb, kp):
    tmp = rgb.copy()
    img = cv2.drawKeypoints(gray, kp, tmp, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    return img

In [5]:
def matcher(kp1, des1, img1, kp2, des2, img2, threshold):
    # BFMatcher with default params
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1,des2, k=2)

    # Apply ratio test
    good = []
    for m,n in matches:
        if m.distance < threshold*n.distance: good.append([m])

    matches = []
    for pair in good:
        matches.append(list(kp1[pair[0].queryIdx].pt + kp2[pair[0].trainIdx].pt))

    matches = np.array(matches)
    return matches

In [6]:
def plot_matches(matches, total_img):
    match_img = total_img.copy()
    offset = total_img.shape[1]/2
    fig, ax = plt.subplots()
    ax.set_aspect('equal')
    ax.imshow(np.array(match_img).astype('uint8')) #　RGB is integer type
    
    ax.plot(matches[:, 0], matches[:, 1], 'xr')
    ax.plot(matches[:, 2] + offset, matches[:, 3], 'xr')
     
    ax.plot([matches[:, 0], matches[:, 2] + offset], [matches[:, 1], matches[:, 3]],
            'r', linewidth=0.5)

    plt.show()

In [7]:
def homography(pairs):
    rows = []
    for i in range(pairs.shape[0]):
        p1 = np.append(pairs[i][0:2], 1)
        p2 = np.append(pairs[i][2:4], 1)
        row1 = [0, 0, 0, p1[0], p1[1], p1[2], -p2[1]*p1[0], -p2[1]*p1[1], -p2[1]*p1[2]]
        row2 = [p1[0], p1[1], p1[2], 0, 0, 0, -p2[0]*p1[0], -p2[0]*p1[1], -p2[0]*p1[2]]
        rows.append(row1)
        rows.append(row2)
    rows = np.array(rows)
    U, s, V = np.linalg.svd(rows)
    H = V[-1].reshape(3, 3)
    H = H/H[2, 2] # standardize to let w*H[2,2] = 1
    return H

In [8]:
def random_point(matches, k=10):
    idx = random.sample(range(len(matches)), k)
    point = [matches[i] for i in idx]
    return np.array(point)

In [9]:
def get_error(points, H):
    num_points = len(points)
    all_p1 = np.concatenate((points[:, 0:2], np.ones((num_points, 1))), axis=1)
    all_p2 = points[:, 2:4]
    estimate_p2 = np.zeros((num_points, 2))
    for i in range(num_points):
        temp = np.dot(H, all_p1[i])
        estimate_p2[i] = (temp/temp[2])[0:2] # set index 2 to 1 and slice the index 0, 1
    # Compute error
    errors = np.linalg.norm(all_p2 - estimate_p2 , axis=1) ** 2

    return errors

In [10]:
def ransac(matches, threshold, iters):
    num_best_inliers = 0
    
    for i in range(iters):
        points = random_point(matches)
        H = homography(points)
        
        #  avoid dividing by zero 
        if np.linalg.matrix_rank(H) < 3: continue
            
        errors = get_error(matches, H)
        idx = np.where(errors < threshold)[0]
        inliers = matches[idx]

        num_inliers = len(inliers)
        if num_inliers > num_best_inliers:
            best_inliers = inliers.copy()
            num_best_inliers = num_inliers
            best_H = H.copy()
            
    print("inliers/matches: {}/{}".format(num_best_inliers, len(matches)))
    return best_inliers, best_H

In [11]:
def stitch_img(left, right, H):
    print("stiching image ...")
    
    # Convert to double and normalize. Avoid noise.
    left = cv2.normalize(left.astype('float'), None, 0.0, 1.0, cv2.NORM_MINMAX)   
    right = cv2.normalize(right.astype('float'), None, 0.0, 1.0, cv2.NORM_MINMAX)   
    
    # left image
    height_l, width_l, channel_l = left.shape
    corners = [[0, 0, 1], [width_l, 0, 1], [width_l, height_l, 1], [0, height_l, 1]]
    corners_new = [np.dot(H, corner) for corner in corners]
    corners_new = np.array(corners_new).T 
    x_news = corners_new[0] / corners_new[2]
    y_news = corners_new[1] / corners_new[2]
    x_min, y_min = min(x_news), min(y_news)
    x_max, y_max = max(x_news), max(y_news)

    translation_mat = np.array([[1, 0, -x_min], [0, 1, -y_min], [0, 0, 1]])
    H = np.dot(translation_mat, H)
    
    height_new = max(int(round(y_max - y_min)), int(round(height_l)))
    width_new = int(round(x_max - x_min) + width_l)
    size = (width_new, height_new)
    warped_l = cv2.warpPerspective(src=left, M=H, dsize=size)
    warped_r = cv2.warpPerspective(src=right, M=translation_mat, dsize=size)
     
    black = np.zeros(3)  # Black pixel.
    black = hex_color(black)
    
    # Stitching procedure, store results in warped_l.
    aa, ai, ma, mi = -10000000, 10000000, -10000000, 10000000
    paa, pai, pma, pmi = 0, 0, 0, 0
    for i in tqdm(range(warped_r.shape[0])):
        for j in range(warped_r.shape[1]):
            pixel_l, pixel_r = warped_l[i, j, :], warped_r[i, j, :] 
            hex_l, hex_r = hex_color(pixel_l), hex_color(pixel_r)

            if not hex_l == black and hex_r == black :
                warped_l[i, j, :] = pixel_l
            elif hex_l == black and not hex_r == black:
                warped_l[i, j, :] = pixel_r
            elif not hex_l == black and not hex_r == black:
                warped_l[i, j, :] = (pixel_l + pixel_r) / 2
                if i + j > aa:
                    aa, paa = i + j, (i, j)
                elif i + j < ai:
                    ai, pai = i + j, (i, j)
                if i - j > ma:
                    ma, pma = i - j, (i, j)
                elif i - j < mi:
                    mi, pmi = i - j, (i, j)
            else:
                pass
    
    stitch_image = warped_l[:warped_r.shape[0], :warped_r.shape[1], :]
    return stitch_image, [pmi, pai, pma, paa]


In [12]:
def removeBlackBorder(img):
    h, w = img.shape[:2]
    reduced_h, reduced_w = 0, w
    # right to left
    for col in range(w - 1, -1, -1):
        all_black = True
        for i in range(h):
            if (np.count_nonzero(img[i, col]) > 0):
                all_black = False
                break
        if (all_black == True):
            reduced_w = reduced_w - 1
            
    # bottom to top 
    for row in range(h):
        all_black = True
        for i in range(reduced_w):
            if (np.count_nonzero(img[row, i]) > 0):
                all_black = False
                break
        if (all_black == True):
            reduced_h += 1
    
    return img[reduced_h:, :reduced_w], reduced_h, reduced_w

In [13]:
if os.path.exists('./410785008') == False: os.mkdir('./410785008')
f = open('./410785008/overlap.txt', 'w', encoding='utf-8')

for i in range(len(os.listdir('left'))):
    picture_index = str(i + 1).zfill(3)
    right_gray, right_origin, right_rgb = read_image('left/' + picture_index + '.jpg')
    right_size = right_rgb.shape
    left_gray, left_origin, left_rgb = read_image('right/' + picture_index + '.jpg', right_size)

    # Better result when using gray
    kp_left, des_left = SIFT(left_gray)
    kp_right, des_right = SIFT(right_gray)

    kp_left_img = plot_sift(left_gray, left_rgb, kp_left)
    kp_right_img = plot_sift(right_gray, right_rgb, kp_right)
    total_kp = np.concatenate((kp_left_img, kp_right_img), axis=1)

    matches = matcher(kp_left, des_left, left_rgb, kp_right, des_right, right_rgb, 0.5)
    total_img = np.concatenate((left_rgb, right_rgb), axis=1)
    #plot_matches(matches, total_img) # Good matches
    inliers, H = ransac(matches, 0.5, 2000)
    #plot_matches(inliers, total_img) # show inliers matches

    pic, points = stitch_img(left_rgb, right_rgb, H)
    pic, crop_y, crop_x = removeBlackBorder(pic)
    print(crop_x, crop_y)
    pic = cv2.flip(pic, 1)
    points = np.array([(i, pic.shape[1] - j) for i, j in points])
    print(points)
    output_string = " ".join([str(j) for i in points for j in i])
    cv2.imwrite("410785008/"+ str(i + 1).zfill(3) + ".jpg", recover_color(pic))
    # pic = cv2.polylines(pic   , [np.flip(points, 1)], True, (255, 0, 0), 2)
    # plt.imshow(pic)

    f.write(str(i + 1).zfill(3) + ' ' + output_string + '\n')
f.close()

inliers/matches: 2571/2669
stiching image ...


  0%|          | 0/1894 [00:00<?, ?it/s]

2574 431
[[ 229  478]
 [ 229 1034]
 [1668 1034]
 [1667  485]]
inliers/matches: 1771/2701
stiching image ...


  0%|          | 0/1783 [00:00<?, ?it/s]

2133 314
[[ 219  642]
 [ 220 1293]
 [1659 1293]
 [1657  644]]
inliers/matches: 535/587
stiching image ...


...

695 64
[[ 47 166]
 [ 48 337]
 [473 337]
 [473 167]]
inliers/matches: 381/389
stiching image ...


  0%|          | 0/540 [00:00<?, ?it/s]

545 0
[[  0 219]
 [  1 354]
 [539 354]
 [539 219]]
inliers/matches: 238/246
stiching image ...


  0%|          | 0/587 [00:00<?, ?it/s]

575 27
[[ 27 107]
 [ 28 210]
 [567 210]
 [567 107]]


In [14]:
for i in range(list(os.listdir('./410785008/'))):
    pic1 = cv2.imread('./left/' + (i+1).zfill(3) + '.jpg')
    pic2 = cv2.imread('./410785008/' + (i+1).zfill(3) + '.jpg')
    print(pic1.shape, pic2.shape)

TypeError: 'list' object cannot be interpreted as an integer